<a href="https://colab.research.google.com/github/t1seo/Python_Notebook/blob/master/effective_python/%ED%81%B4%EB%9E%98%EC%8A%A4%EC%99%80%20%EC%9D%B8%ED%84%B0%ED%8E%98%EC%9D%B4%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 5 클래스와 인터페이스

## BETTER WAY 37 내장 타입을 여러 단계로 내포시키기보다는 클래스를 합성하라




## BETTER WAY 38 간단한 인터페이스의 경우 클래스 대신 함수를 받아라

API가 실행되는 과정에서 전달한 함수를 실행하는 경우, 이런 함수를 **훅(hook)**이라 부른다.

파이썬에서는 단순히 인자와 반환 값이 잘 정의된, 상태가 없는 함수를 훅으로 사용하는 경우가 많다. 

함수는 클래스보다 정의하거나 기술하기가 더 쉬우므로 훅으로 사용하기에는 함수가 이상적이다.

또한, 파이썬은 함수를 **일급 시민 객체**로 취급하기 때문에 함수를 훅으로 사용할 수 있다. 

함수나 메서드가 일급 시민 객체라는 말은 파이썬 언어에서 사용할 수 있는 다른 일반적인 값과 마찬가지로 함수나 메서드를 다른 함수(또는 메서드)에 넘기거나 변수 등으로 참조할 수 있다는 의미이다.

In [ ]:
names = ['소크라테스', '아르키메데스', '플라톤', '아리스토텔레스']
names.sort(key=len) # key 훅으로 len 내장 함수 전달
print(names)

### defaultdict 클래스의 동작을 사용자 정의

`defaultdict`에는 딕셔너리 안에 없는 키에 접근할 경우 호출되는 인자가 없는 함수를 전달할 수 있다.

In [ ]:
def log_missing():
    print('키 추가됨')
    return 0

from collections import defaultdict

current = {'초록': 12, '파랑': 3}
increments = [
    ('빨강', 5),
    ('파랑', 17),
    ('주황', 9),
]

result = defaultdict(log_missing, current) # log_missing 함수를 훅으로
print('이전:', dict(result))
for key, amount in increments:
    result[key] += amount # 키 추가될 때마다 log_missing 함수 실행
print('이후:', dict(result))

- 존재하지 않는 키에 접근할 때 로그를 남기고 0을 디폴트 값으로 반환한다.

### defaultdict에 전달하는 디폴트 값 훅이 존재하지 않는 키에 접근한 총횟수 세기

In [ ]:
def increment_with_report(current, increments):
    added_count = 0

    def missing(): # 클로저가 있는 도우미 함수
        nonlocal added_count  # 상태가 있는 클로저
        added_count += 1
        return 0

    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount

    return result, added_count

result, count = increment_with_report(current, increments)
print("Count: ", count)
print(dict(result))
# assert count == 2

- 인터페이스에서 간단한 함수를 인자로 받으면 클로저 안에 상태를 감추능 기능 계층을 쉽게 추가할 수 있다.
- 하지만 상태를 다루기 위한 훅으로 클로저를 사용하면 상태가 없는 함수에 비해 읽고 이해하기 어렵다.

### 추적하고 싶은 상태를 저장하는 작은 클래스 정의

In [ ]:
class CountMissing:
    def __init__(self):
        self.added = 0 # 키에 접근한 횟수

    def missing(self): # missing 메서드
        self.added += 1 # 키에 접근한 횟수 증가
        return 0 # 기본 value 값

counter = CountMissing()
result = defaultdict(counter.missing, current) # CounterMissing.missing 메서드를 훅으로
for key, amount in increments:
    result[key] += amount

print("Count: ", count)
print(dict(result))
assert counter.added == 2

### __call__ 특별 메서드 사용


위의 경우를 좀 더 명확히 표헌하기 위해 파이썬에서는 클래스에 `__call__` 특별 메서드를 정의할 수 있다.

`__call__`을 사용하면 객체를 함수처럼 호출할 수 있다. 그리고 `__call__`이 정의된 클래스의 인스턴스에 대해 `callable` 내장 함수를 호출하면, 다른 일반 함수나 메서드와 마찬가지로 `True`가 반환된다. 

이런 방식으로 정의돼서 호출될 수 있는 모든 객체를 **호출 가능(callable) 객체)**라고 부른다.


In [ ]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0

    def __call__(self): # 특별 메서드
        self.added += 1
        return 0


counter = BetterCountMissing()
assert counter() == 0
assert callable(counter)

#
counter = BetterCountMissing()
result = defaultdict(counter, current) # __call__에 의존함
for key, amount in increments:
    result[key] += amount

print("Count: ", count)
print(dict(result))
# assert counter.added == 2

- `__call__` 메서드는 (API 훅처럼) 함수가 인자로 쓰일 수 있는 부분에서 이 클래스가 인스턴스를 상요할 수 있다는 사실을 나타낸다.